In [26]:
import time
import threading
import warnings
from datetime import datetime
from typing import Dict , Optional
import pandas as pd # type: ignore
warnings.filterwarnings('ignore')

# interactive broker API live

In [ ]:
# !pip install ibapi
# !pip install ib_insync

DEPRECATION: Loading egg at /Users/leopn/.pyenv/versions/3.11.13/envs/stock/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
DEPRECATION: Loading egg at /Users/leopn/.pyenv/versions/3.11.13/envs/stock/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [27]:
from ibapi.client import EClient # type: ignore
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import BarData

In [28]:
class TradingApp(EClient,EWrapper):
    def __init__(self)->None:
        EClient.__init__(self,self)
        self.data: Dict[int,pd.DataFrame]={}
    def error(self,reqId:int, erroCode:int,errorString:str)->None:
        print(f'Error: ',{reqId},{erroCode},{errorString})
    def get_historical_data(self,reqId:int,contract:Contract)->pd.DataFrame:
        self.data[reqId]=pd.DataFrame(columns=['time','high','low','close'])
        self.data[reqId].set_index('time',inplace=True)
        self.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='MIDPOINT',
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[],
        )
        time.sleep(4)
        return self.data[reqId]
    def historicalData(self, reqId:int, bar:BarData)->None:
        df=self.data[reqId]
        df.loc[pd.to_datetime(bar.date,unit='s'),['high','low','close']]=[bar.high,bar.low,bar.close]
        df=df.astype(float)
        self.data[reqId]=df
    @staticmethod
    def get_stock_contract(symbol:str)->Contract:
        contract=Contract()
        contract.symbol=symbol
        contract.secType='STK'
        contract.exchange='SMART'
        contract.currency='USD'
        return contract
        
    
        


# you need to have TWS API and Trader WorkStation

In [29]:
app=TradingApp()


In [30]:
app.connect('127.0.0.1',7497,clientId=6)

In [31]:
threading.Thread(target=app.run,daemon=True).start()

Exception in thread Thread-8 (run):
Traceback (most recent call last):
  File "/Users/leopn/.pyenv/versions/3.11.13/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/leopn/.pyenv/versions/3.11.13/envs/stock/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/leopn/.pyenv/versions/3.11.13/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/leopn/.pyenv/versions/3.11.13/envs/stock/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg/ibapi/client.py", line 398, in run
  File "/Users/leopn/.pyenv/versions/3.11.13/envs/stock/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg/ibapi/decoder.py", line 1506, in interpret
  File "/Users/leopn/.pyenv/versions/3.11.13/envs/stock/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg/ibapi/decoder.py", line 1388, in processErrorMsg
TypeError: TradingApp.error() takes 4 positiona

In [23]:
nvda=TradingApp.get_stock_contract('NVDA')

In [32]:
nvda

4939979600: 0,NVDA,STK,,,0,,,SMART,,USD,,,False,,,,combo:

In [33]:
data=app.get_historical_data(10,nvda)
data

Error:  {10} {504} {'Not connected'}


,high,low,close
time,,,


## These are most useful libraries in python for indicators and chart for data . 
### pip install mplfinance for renko and kagi
## pip install pandas_ta for (indicator like MACD,RSI ATR,..) https://github.com/twopirllc/pandas-ta.git
# pip install plotly  for better visualization, like Kagi and Renko